In [1]:
### To install dependencies and run this notebook:

# mkdir ropsten_revival
# cd ropsten_revival
# pipenv --two
# pipenv install git+https://github.com/ethereum/pyethereum@eip155#egg=ethereum
# pipenv install bitcoin
# pipenv install jupyter
# pipenv shell
# jupyter notebook

In [2]:
import json
import random

import rlp
from ethereum.blocks import BlockHeader
from ethereum.transactions import Transaction

#from ethereum.transactions import EIP155Transaction as Transaction
# on EIP-155 branch https://github.com/ethereum/pyethereum/tree/eip155
# but broken


In [3]:
f = open('/Users/macbook/eth_devops/ropsten-dos-chain.rlp', 'r')
outdata = []
RESCUE_FORK_BLOCK = 585000
LAST_BLOCK = 722677 # this depends on your RLP chain export


In [4]:
blknum, pos = 0, 0

txs_to_replay = []
tx_total_count = 0
while 1:
    f.seek(pos)
    prefix = f.read(10)
    _typ, _len, _pos = rlp.codec.consume_length_prefix(prefix, 0)
    blkdata = prefix + f.read(_pos + _len - 10)
    header = rlp.decode(rlp.descend(blkdata, 0), BlockHeader)
    pos += _pos + _len
    if header.number % 50000 == 0:
        print "#", header.number
    if header.number <= RESCUE_FORK_BLOCK:
        continue
    txs_data = rlp.decode(rlp.descend(blkdata, 1))
    txcount = len(txs_data)
    tx_total_count += txcount
    for tx in txs_data:
        txs_to_replay.append(rlp.encode(tx).encode('hex'))
    #if header.number > 720000:
    #    print header.number # 722677 is the last block in the export   
    if header.number == LAST_BLOCK:
        break

print "all blocks scraped. total tx count:", tx_total_count

# 50000
# 100000
# 150000
# 200000
# 250000
# 300000
# 350000
# 400000
# 450000
# 500000
# 550000
# 600000
# 650000
# 700000
all blocks scraped. total tx count: 260852


In [5]:
len(txs_to_replay)

260852

In [6]:
# Some views of raw tx's

In [7]:
txs_to_replay[0]

'f8708304c04385746a5288008304cb2694dcd6a8e0419574f529d3726b8e7f5222877102e9880de0b6b3a7640000801ba09a8c1eb3710fdba4250e44c9e4a93f0a1aaa4b47876253563e64684a2fdfdec4a03acde787864c9569d557aad6ed8687751f58132a3603e8f43024dcef51b3f6c9'

In [8]:
txs_to_replay[0].decode('hex')

'\xf8p\x83\x04\xc0C\x85tjR\x88\x00\x83\x04\xcb&\x94\xdc\xd6\xa8\xe0A\x95t\xf5)\xd3rk\x8e\x7fR"\x87q\x02\xe9\x88\r\xe0\xb6\xb3\xa7d\x00\x00\x80\x1b\xa0\x9a\x8c\x1e\xb3q\x0f\xdb\xa4%\x0eD\xc9\xe4\xa9?\n\x1a\xaaKG\x87bSV>dhJ/\xdf\xde\xc4\xa0:\xcd\xe7\x87\x86L\x95i\xd5W\xaa\xd6\xed\x86\x87u\x1fX\x13*6\x03\xe8\xf40$\xdc\xefQ\xb3\xf6\xc9'

In [9]:
rlp.decode(txs_to_replay[0].decode('hex'))

['\x04\xc0C',
 'tjR\x88\x00',
 '\x04\xcb&',
 '\xdc\xd6\xa8\xe0A\x95t\xf5)\xd3rk\x8e\x7fR"\x87q\x02\xe9',
 '\r\xe0\xb6\xb3\xa7d\x00\x00',
 '',
 '\x1b',
 '\x9a\x8c\x1e\xb3q\x0f\xdb\xa4%\x0eD\xc9\xe4\xa9?\n\x1a\xaaKG\x87bSV>dhJ/\xdf\xde\xc4',
 ':\xcd\xe7\x87\x86L\x95i\xd5W\xaa\xd6\xed\x86\x87u\x1fX\x13*6\x03\xe8\xf40$\xdc\xefQ\xb3\xf6\xc9']

In [10]:
[field.encode('hex') for field in rlp.decode(txs_to_replay[0].decode('hex'))]

['04c043',
 '746a528800',
 '04cb26',
 'dcd6a8e0419574f529d3726b8e7f5222877102e9',
 '0de0b6b3a7640000',
 '',
 '1b',
 '9a8c1eb3710fdba4250e44c9e4a93f0a1aaa4b47876253563e64684a2fdfdec4',
 '3acde787864c9569d557aad6ed8687751f58132a3603e8f43024dcef51b3f6c9']

In [11]:
tx1 = rlp.decode(txs_to_replay[0].decode('hex'), Transaction)

In [12]:
tx1.fields

[('nonce', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('gasprice', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('startgas', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('to', <rlp.sedes.binary.Binary at 0x10686c510>),
 ('value', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('data', <rlp.sedes.binary.Binary at 0x1067da850>),
 ('v', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('r', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>),
 ('s', <rlp.sedes.big_endian_int.BigEndianInt at 0x1067a9b50>)]

In [13]:
tx1.sender.encode('hex') 

/Users/macbook/.virtualenvs/ropsten_rescue-HldH1mX1/lib/python2.7/site-packages/secp256k1/__init__.py:228: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  self.ctx, res_compressed, outlen, self.public_key, compflag)


'687422eea2cb73b5d3e242ba5456b782919afc85'

In [14]:
tx1.hash.encode('hex')

'38f6b069969b4fccecc603ac4ad1449737290de570c0aa642efad23d0cfda47e'

In [15]:
tx1.startgas

314150

In [16]:
txs_to_replay[473]

'f8d2138504a817c800833d090094529631401334eb94a123bf84e38231b56576a18b8802386f26fc100000b86495695a440000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000012aa09073c9cec6fb26b59e86288bc2ba8963c5d5159afd2b099f8b7efba9b9c486e4a06138b0e33a2c1b1f3543a93aab00c178524c2cdf61d0f694d21353a92fee41ca'

In [17]:
eip_155_tx_obj = rlp.decode(txs_to_replay[473].decode('hex'), Transaction)

In [18]:
eip_155_tx_obj.nonce

19

In [19]:
eip_155_tx_obj.hash.encode('hex')

'42f79eeff40dcb5a972010d574a6542a9195cac1ddf623a70ad4fbe39cf15e8f'

In [20]:
eip_155_tx_obj.sender # EIP155 branch not quite there yet

InvalidTransaction: Invalid signature values!

In [21]:
# loop through and filter attack tx's with startgas over 5 million

In [22]:
default_gas_price = 20000000000
low_price_txs = [] # low price tx's won't be replayed unless mining node accepts them
good_txs = []
excluded_hashes = []
excluded_count = 0
total_gas = 0
total_gas_including_bad = 0

i = 0
for tx_hex in txs_to_replay:
    i += 1
    if i % 10000 == 0:
        print "doing tx i:", i
    tx_obj = rlp.decode(tx_hex.decode('hex'), Transaction)
    total_gas_including_bad += tx_obj.startgas
    if tx_obj.startgas < 5000000: # good tx's have startgas less than 5 million
        good_txs.append(tx_hex)
        total_gas += tx_obj.startgas
        if tx_obj.gasprice < default_gas_price:
            low_price_txs.append([tx_obj.gasprice, tx_obj.hash.encode('hex')])
       
    else:
        # attacker tx's have startgas greater than 5 million
        excluded_count += 1 
        excluded_hashes.append(tx_obj.hash.encode('hex')) 
        #sender = tx_obj.sender.encode('hex') # breaks on EIP-155

print "all done."

doing tx i: 10000
doing tx i: 20000
doing tx i: 30000
doing tx i: 40000
doing tx i: 50000
doing tx i: 60000
doing tx i: 70000
doing tx i: 80000
doing tx i: 90000
doing tx i: 100000
doing tx i: 110000
doing tx i: 120000
doing tx i: 130000
doing tx i: 140000
doing tx i: 150000
doing tx i: 160000
doing tx i: 170000
doing tx i: 180000
doing tx i: 190000
doing tx i: 200000
doing tx i: 210000
doing tx i: 220000
doing tx i: 230000
doing tx i: 240000
doing tx i: 250000
doing tx i: 260000
all done.


In [23]:
total_gas

233919249349

In [24]:
excluded_count

9278

In [25]:
total_gas_including_bad

532306863342

In [26]:
len(low_price_txs)

142

In [27]:
len(good_txs)

251574

In [28]:
txs_to_replay = good_txs

In [29]:
def sumGas(tx_list):
    total_gas = 0
    for tx_hex in tx_list:
        tx_obj = rlp.decode(tx_hex.decode('hex'), Transaction)
        total_gas += tx_obj.startgas
    return total_gas

In [30]:
# saving the tx's into batches, otherwise its one huge json file
# manually choosing batch sizes based on total startgas and file size

In [31]:
sumGas(txs_to_replay[0:25000])

5633707563

In [32]:
with open('txs_batch_0_to_25k.json', 'w') as f:
    json.dump(txs_to_replay[0:25000], f)

In [33]:
# starting at 24999 instead of 25000 will replay first tx twice
# replaying multiple times is no problem, will just get a 'nonce too low' error from the node
sumGas(txs_to_replay[24999:50000]) 

19591387570

In [34]:
with open('txs_batch_25k_to_50k.json', 'w') as f:
    json.dump(txs_to_replay[24999:50000], f)

In [35]:
sumGas(txs_to_replay[49999:60000])

8795063601

In [36]:
with open('txs_batch_50k_to_60k.json', 'w') as f:
    json.dump(txs_to_replay[49999:60000], f)

In [37]:
sumGas(txs_to_replay[59999:70000])

9682211635

In [38]:
with open('txs_batch_60k_to_70k.json', 'w') as f:
    json.dump(txs_to_replay[59999:70000], f)

In [39]:
sumGas(txs_to_replay[69999:75000])

2120194297

In [40]:
with open('txs_batch_70k_to_75k.json', 'w') as f:
    json.dump(txs_to_replay[69999:75000], f)

In [41]:
sumGas(txs_to_replay[74999:100000])

21796283894

In [42]:
with open('txs_batch_75k_to_100k.json', 'w') as f:
    json.dump(txs_to_replay[74999:100000], f)

In [43]:
sumGas(txs_to_replay[99999:150000])

64249856087

In [44]:
with open('txs_batch_100k_to_150k.json', 'w') as f:
    json.dump(txs_to_replay[99999:150000], f)

In [45]:
with open('txs_batch_150k_to_160k.json', 'w') as f:
    json.dump(txs_to_replay[149999:160000], f)

In [46]:
with open('txs_batch_160k_to_170k.json', 'w') as f:
    json.dump(txs_to_replay[159999:170000], f)

In [47]:
with open('txs_batch_170k_to_180k.json', 'w') as f:
    json.dump(txs_to_replay[169999:180000], f)

In [48]:
with open('txs_batch_180k_to_190k.json', 'w') as f:
    json.dump(txs_to_replay[179999:190000], f)

In [49]:
with open('txs_batch_190k_to_220k.json', 'w') as f:
    json.dump(txs_to_replay[189999:220000], f)

In [50]:
with open('txs_batch_220k_to_250k+.json', 'w') as f:
    json.dump(txs_to_replay[219999:], f)

In [51]:
# or automate the batch sizes

In [52]:
BATCH_SIZE = 1000000 * 25 # 25 megabyte json files

batch_i = 1
current_batch_len = 0
tx_batch = []
for tx in txs_to_replay:
    tx_batch.append(tx)
    current_batch_len += len(tx)
    if current_batch_len > BATCH_SIZE:
        file_name = 'ropsten_rescue_txs_' + str(batch_i) + '.json'
        with open(file_name, 'w') as f:
            json.dump(tx_batch, f)
            print "wrote", file_name
        tx_batch = []
        current_batch_len = 0
        batch_i += 1

wrote ropsten_rescue_txs_1.json
wrote ropsten_rescue_txs_2.json
wrote ropsten_rescue_txs_3.json
wrote ropsten_rescue_txs_4.json
wrote ropsten_rescue_txs_5.json
wrote ropsten_rescue_txs_6.json
wrote ropsten_rescue_txs_7.json
wrote ropsten_rescue_txs_8.json
wrote ropsten_rescue_txs_9.json
wrote ropsten_rescue_txs_10.json
wrote ropsten_rescue_txs_11.json
wrote ropsten_rescue_txs_12.json
wrote ropsten_rescue_txs_13.json
